# Assignment 3

First import all the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

Load datasets?

In [2]:
df_houseprices_testdata = pd.read_csv('test.csv', sep=',')
df_houseprices_traindata = pd.read_csv('train.csv', sep=',')
df_houseprices_traindata.set_index(["Id"]).head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Look at the data

In [3]:
df_houseprices_testdata.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
pd.set_option('display.max_rows', len(df_houseprices_traindata["Id"]))
df_houseprices_traindata.T

,0,1,2,3,4,5,6,7,8,9,...,1450,1451,1452,1453,1454,1455,1456,1457,1458,1459
Id,1,2,3,4,5,6,7,8,9,10,...,1451,1452,1453,1454,1455,1456,1457,1458,1459,1460
MSSubClass,60,20,60,70,60,50,20,60,50,190,...,90,20,180,20,20,60,20,70,20,20
MSZoning,RL,RL,RL,RL,RL,RL,RL,RL,RM,RL,...,RL,RL,RM,RL,FV,RL,RL,RL,RL,RL
LotFrontage,65,80,68,60,84,85,75,NaN,51,50,...,60,78,35,90,62,62,85,66,68,75
LotArea,8450,9600,11250,9550,14260,14115,10084,10382,6120,7420,...,9000,9262,3675,17217,7500,7917,13175,9042,9717,9937
Street,Pave,Pave,Pave,Pave,Pave,Pave,Pave,Pave,Pave,Pave,...,Pave,Pave,Pave,Pave,Pave,Pave,Pave,Pave,Pave,Pave
Alley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Pave,NaN,NaN,NaN,NaN,NaN
LotShape,Reg,Reg,IR1,IR1,IR1,IR1,Reg,IR1,Reg,Reg,...,Reg,Reg,Reg,Reg,Reg,Reg,Reg,Reg,Reg,Reg
LandContour,Lvl,Lvl,Lvl,Lvl,Lvl,Lvl,Lvl,Lvl,Lvl,Lvl,...,Lvl,Lvl,Lvl,Lvl,Lvl,Lvl,Lvl,Lvl,Lvl,Lvl
Utilities,AllPub,AllPub,AllPub,AllPub,AllPub,AllPub,AllPub,AllPub,AllPub,AllPub,...,AllPub,AllPub,AllPub,AllPub,AllPub,AllPub,AllPub,AllPub,AllPub,AllPub


In [5]:
fit_cols = ["LotArea","PoolArea","YrSold","GarageArea","OverallQual","OverallCond","BedroomAbvGr","YearBuilt","Fireplaces"]

Now I need to figure out what to focus on, I might want the fields:
    - (YearRemodAdd - YrSold)*-1 = delta_years_mod_and_sold
    - LotArea OK
    - OverallCond OK
    - OverallQual OK
    - MSZoning OK

In [6]:
df_houseprices_traindata["MSZoning"].describe()

count     1460
unique       5
top         RL
freq      1151
Name: MSZoning, dtype: object

It is obvious that there is many low density residential zoned houses, I'll now separate the different MSZones to their own coloumn

In [7]:
MSZones_seperated = pd.get_dummies(df_houseprices_traindata["MSZoning"], prefix="MSZone", drop_first=True, dtype=int)
MSZones_seperated.head()

,MSZone_FV,MSZone_RH,MSZone_RL,MSZone_RM
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0


Now that we have separated them into coloumns, we can select the other Quality, condition and LotArea coloumns. 
Then we join them together, we now just need the last coloumn; "delta_years_mod_and_sold"

In [8]:
df_Choosen_numeric_DataColoumns = pd.DataFrame(df_houseprices_traindata[["OverallQual","LotArea","OverallCond"]])
df_all_but_yearDelta_coloumns = df_Choosen_numeric_DataColoumns.join(MSZones_seperated)
df_all_but_yearDelta_coloumns.head()

,OverallQual,LotArea,OverallCond,MSZone_FV,MSZone_RH,MSZone_RL,MSZone_RM
0,7,8450,5,0,0,1,0
1,6,9600,8,0,0,1,0
2,7,11250,5,0,0,1,0
3,7,9550,5,0,0,1,0
4,8,14260,5,0,0,1,0


We calculate the delta_years_mod_and_sold like so: (YearRemodAdd - YrSold)*-1

In [9]:
yearRemodAdd = df_houseprices_traindata["YearRemodAdd"]
yearsold = df_houseprices_traindata["YrSold"]
calculatedCols = (yearRemodAdd-yearsold) * -1
df_delta_years_mod_and_sold = pd.DataFrame(calculatedCols,columns=["delta_years_mod_and_sold"])
df_delta_years_mod_and_sold.head()

,delta_years_mod_and_sold
0,5
1,31
2,6
3,36
4,8


In [10]:
finaldataset = df_all_but_yearDelta_coloumns.join(df_delta_years_mod_and_sold)
finaldataset.head()

,OverallQual,LotArea,OverallCond,MSZone_FV,MSZone_RH,MSZone_RL,MSZone_RM,delta_years_mod_and_sold
0,7,8450,5,0,0,1,0,5
1,6,9600,8,0,0,1,0,31
2,7,11250,5,0,0,1,0,6
3,7,9550,5,0,0,1,0,36
4,8,14260,5,0,0,1,0,8


## Now well try to import, instanciate, fit predict!!

In [11]:
from sklearn.linear_model import LinearRegression
myLinearReg = LinearRegression()
X1 = finaldataset
y1 = df_houseprices_traindata.SalePrice

myLinearReg.fit(X1,y1)
y1_predicted = myLinearReg.predict(X1)
df_y1_predicted = pd.DataFrame(y1_predicted, columns=["SalePrice"])
df_y1_predicted.head()

,SalePrice
0,227166.807249
1,175140.712467
2,230398.977371
3,214924.297184
4,273344.217577


Now we'll try to estimate the quality of our predictions

In [12]:
from sklearn import metrics
metrics.mean_squared_error( df_houseprices_traindata.SalePrice, df_y1_predicted.SalePrice )

2012810644.1397839

That was.. Bad ... we think?
Let's try remove some columns and see what that does to our prediction quality
- Remove delta year: got worse!
- Remove delta year and MSZones: Worse!
- Add Salecondition: Slightly better!
- Add Neighboorhood: A lot better!

In [13]:
df_SaleCondition_seperated = pd.get_dummies(df_houseprices_traindata["SaleCondition"], prefix="SaleCondition", drop_first=True, dtype=int)
df_neighboorhood_seperated = pd.get_dummies(df_houseprices_traindata["Neighborhood"], prefix="Neighborhood", drop_first=True, dtype=int)

In [14]:
# add coloumns
X3 = df_houseprices_traindata[fit_cols]

In [15]:
# added our own custom coloumns
X3=X3.join(df_SaleCondition_seperated).join(df_neighboorhood_seperated).join(df_delta_years_mod_and_sold)

In [16]:
X3 = df_houseprices_traindata[fit_cols]
y2 = df_houseprices_traindata.SalePrice
myLinearReg.fit(X3,y2)
y2_predicted = myLinearReg.predict(X3)
df_y2_predicted = pd.DataFrame(y2_predicted, columns=["SalePrice"])
df_y2_predicted.head()

,SalePrice
0,210543.704481
1,194006.503627
2,234576.520402
3,208362.582195
4,292948.663640


In [17]:
metrics.mean_squared_error( df_houseprices_traindata.SalePrice, df_y2_predicted.SalePrice )
np.sqrt(1754246692)

41883.72824856928

We noticed that salecondition, might have something to do with when were really off in our predictions

In [18]:
# from pandas.plotting import scatter_matrix
# scatter_matrix(df_houseprices_traindata, alpha=0.2, figsize=(50,50), diagonal='kde')    

In [19]:
df_houseprices_traindata.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0


In [20]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
df_dataset_ridge = df_houseprices_traindata.drop(["SalePrice"], 1)
df_dataset_ridge.dropna(axis="columns")
df_dataset_ridge.drop?

In [21]:
df_dataset_ridge.columns[df_dataset_ridge.dtypes != 'object']

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [22]:
df_dataset_ridge.dtypes != 'object'

Id                True
MSSubClass        True
MSZoning         False
LotFrontage       True
LotArea           True
Street           False
Alley            False
LotShape         False
LandContour      False
Utilities        False
LotConfig        False
LandSlope        False
Neighborhood     False
Condition1       False
Condition2       False
BldgType         False
HouseStyle       False
OverallQual       True
OverallCond       True
YearBuilt         True
YearRemodAdd      True
RoofStyle        False
RoofMatl         False
Exterior1st      False
Exterior2nd      False
MasVnrType       False
MasVnrArea        True
ExterQual        False
ExterCond        False
Foundation       False
BsmtQual         False
BsmtCond         False
BsmtExposure     False
BsmtFinType1     False
BsmtFinSF1        True
BsmtFinType2     False
BsmtFinSF2        True
BsmtUnfSF         True
TotalBsmtSF       True
Heating          False
HeatingQC        False
CentralAir       False
Electrical       False
1stFlrSF   